In [1]:
import numpy as np
import torch
from torchvision import transforms
from torchvision.ops import sigmoid_focal_loss


import torch.nn as nn
import torch.nn.functional as F


from torch.autograd import Variable

# https://d2l.ai/chapter_recurrent-modern/gru.html

import matplotlib.pyplot as plt

In [2]:
shape = (300, 180, 180,3)
tensor = torch.randint(0, 7, shape)


In [3]:
def get_window_index(tensor): 


    tensor_count = np.count_nonzero(tensor[:,:,:,1], axis = 0) 
    min_events = 250

    min_events_index = np.where(tensor_count >= min_events) # number of events so >= 1 or > 0 is the same as np.nonzero

    min_events_row = min_events_index[0]
    min_events_col = min_events_index[1]

    min_events_indx = [(row, col) for row, col in zip(min_events_row, min_events_col)] 

    indx = min_events_indx[np.random.choice(len(min_events_indx))] # dumb but working solution of np.random instead of random

    # if you want a random temporal window, it is here.
    window_index = {'row_indx':indx[0], 'col_indx':indx[1]} 

    return(window_index)


In [6]:
def get_window_coords(window_index):

    # you can change this back to random if you want
    window_dim = 32#config.window_dim

    # Randomly select a window around the sampled index. np.clip is used to ensure that the window does not go out of bounds
    min_row_indx = np.clip(int(window_index['row_indx'] - np.random.randint(0, window_dim)), 0, 180 - window_dim)
    max_row_indx = min_row_indx + window_dim
    min_col_indx = np.clip(int(window_index['col_indx'] - np.random.randint(0, window_dim)), 0, 180 - window_dim)
    max_col_indx = min_col_indx + window_dim

    # make dict of window coords to return
    window_coords = {
        'min_row_indx':min_row_indx, 
        'max_row_indx':max_row_indx, 
        'min_col_indx':min_col_indx, 
        'max_col_indx':max_col_indx, 
        'dim':window_dim}

    return(window_coords)

In [9]:
for i in range(100):

    window_index = get_window_index(tensor)
    window_coords = get_window_coords(window_index)

    if window_coords['min_row_indx'] < 0 or window_coords['min_col_indx'] < 0 or window_coords['max_row_indx'] > 180 or window_coords['max_col_indx'] > 180:
        print('error1')

    if window_coords['max_row_indx'] - window_coords['min_row_indx'] != 32 or window_coords['max_col_indx'] - window_coords['min_col_indx'] != 32:
        print('error2')

    print(window_coords)
    print(window_coords['max_row_indx'] - window_coords['min_row_indx'])
    print(window_coords['max_col_indx'] - window_coords['min_col_indx'])

{'min_row_indx': 13, 'max_row_indx': 45, 'min_col_indx': 99, 'max_col_indx': 131, 'dim': 32}
32
32
{'min_row_indx': 148, 'max_row_indx': 180, 'min_col_indx': 106, 'max_col_indx': 138, 'dim': 32}
32
32
{'min_row_indx': 71, 'max_row_indx': 103, 'min_col_indx': 0, 'max_col_indx': 32, 'dim': 32}
32
32
{'min_row_indx': 0, 'max_row_indx': 32, 'min_col_indx': 115, 'max_col_indx': 147, 'dim': 32}
32
32
{'min_row_indx': 102, 'max_row_indx': 134, 'min_col_indx': 118, 'max_col_indx': 150, 'dim': 32}
32
32
{'min_row_indx': 142, 'max_row_indx': 174, 'min_col_indx': 17, 'max_col_indx': 49, 'dim': 32}
32
32
{'min_row_indx': 104, 'max_row_indx': 136, 'min_col_indx': 58, 'max_col_indx': 90, 'dim': 32}
32
32
{'min_row_indx': 0, 'max_row_indx': 32, 'min_col_indx': 59, 'max_col_indx': 91, 'dim': 32}
32
32
{'min_row_indx': 63, 'max_row_indx': 95, 'min_col_indx': 148, 'max_col_indx': 180, 'dim': 32}
32
32
{'min_row_indx': 18, 'max_row_indx': 50, 'min_col_indx': 13, 'max_col_indx': 45, 'dim': 32}
32
32
{'min

In [8]:
window_coords

{'min_row_indx': 141,
 'max_row_indx': 173,
 'min_col_indx': 118,
 'max_col_indx': 150,
 'dim': 32}